In [1]:
import os

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers, optimizers
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
# Version check

import sklearn
import keras
import tensorflow as tf

print('Sci-kit learn version :' , sklearn.__version__)
print('Keras version :' , keras.__version__)
print('With tensorflow backend, version : ', tf.__version__ )

Sci-kit learn version : 0.20.3
Keras version : 2.3.1
With tensorflow backend, version :  1.14.0


In [3]:
points = np.load("points.npy")
rotations = np.load("rotations.npy")
print(points.shape, rotations.shape)

(1000, 93, 3) (1000, 3)


In [4]:
print(points.shape, points[0][:15], points[0].min(), points[0].max())

(1000, 93, 3) [[-0.48876673  0.59847438  1.64122784]
 [-1.7565465   0.44131052  0.10238612]
 [ 0.71782744  1.47511649  0.5576387 ]
 [-0.54995227  1.31795263 -0.98120302]
 [-1.62624359  0.53598106 -0.01463312]
 [-0.35846394  0.69314492  1.52420855]
 [-0.37847868  1.33920979 -0.773067  ]
 [-1.58507288  0.46256772  0.31052214]
 [-0.66024029  0.57721722  1.43309188]
 [ 0.20924175 -0.51396441  1.17978609]
 [-1.01736724 -0.74454165 -0.03390032]
 [-1.34430301  0.32235277  0.12648338]
 [-0.11255008  0.01617223 -1.20860577]
 [-0.8870644  -0.64987111 -0.15091959]
 [ 0.0789389  -0.60863495  1.29680538]] -1.7934601306915283 3.3396642208099365


In [5]:
points[0], points[50]

(array([[-0.48876673,  0.59847438,  1.64122784],
        [-1.7565465 ,  0.44131052,  0.10238612],
        [ 0.71782744,  1.47511649,  0.5576387 ],
        [-0.54995227,  1.31795263, -0.98120302],
        [-1.62624359,  0.53598106, -0.01463312],
        [-0.35846394,  0.69314492,  1.52420855],
        [-0.37847868,  1.33920979, -0.773067  ],
        [-1.58507288,  0.46256772,  0.31052214],
        [-0.66024029,  0.57721722,  1.43309188],
        [ 0.20924175, -0.51396441,  1.17978609],
        [-1.01736724, -0.74454165, -0.03390032],
        [-1.34430301,  0.32235277,  0.12648338],
        [-0.11255008,  0.01617223, -1.20860577],
        [-0.8870644 , -0.64987111, -0.15091959],
        [ 0.0789389 , -0.60863495,  1.29680538],
        [-0.39888471,  0.40029582,  1.41163039],
        [ 1.15522957,  0.17333612,  0.33023602],
        [ 0.01775342,  0.11084327, -1.32562542],
        [-0.71716946,  1.30177104, -0.84178793],
        [-1.66315746,  0.6144706 ,  0.00776213],
        [-0.3953777 

In [6]:
# Normalize every 
def normalize_points(arr):
    result = np.interp(arr, (arr.min(), arr.max()), (0, 1))
    #result = np.sort(result, axis=0)
    return result.flatten()

#points_min = points.min(axis=2).reshape(rows_count, row_length)
#points_max = points.min(axis=2)
#points_max = np.tile(points_max, 3).reshape(rows_count, row_length, 3)
#points_normalized = points - points_min
#normalize_points(points[0])

X = np.empty([points.shape[0], points.shape[1]*3])

for row in range(points.shape[0]):
    X[row] = normalize_points(points[row])
    
#points_normalized = np.apply_along_axis(normalize_points, 2, points)

print(X.shape, X[0][:15], X[0].min(), X[0].max())

(1000, 279) [0.2541714  0.46598024 0.6691223  0.00719126 0.43536266 0.36933573
 0.48923178 0.63676163 0.45802491 0.24225165 0.60614405 0.15823835
 0.03257598 0.45380572 0.34653885] 0.0 1.0


In [7]:
y = rotations
print(X.shape, rotations.shape)

(1000, 279) (1000, 3)


In [8]:
def gen_model_1():
    model = Sequential()
    model.add(Dense(279, input_dim=279))
    model.add(Dense(70, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def gen_model_2():
    model = Sequential()
    model.add(Dense(279, input_dim=279))
    model.add(Dense(150, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(Dense(70, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def gen_model_3():
    model = Sequential()
    model.add(Dense(279, input_dim=279))
    model.add(Dense(70, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(Dense(150, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

models = [gen_model_1, gen_model_2, gen_model_3]

In [9]:
#set seed for reproducability 
seed = 7

# Add a data Scaler and the keras regressor containing our model function to a list of estimators

    
#add our estimator list to a Sklearn pipeline

 
#initialize instance of k-fold validation from sklearn api

kfold = KFold(n_splits=5, random_state=seed)

In [11]:
for m in models:
    pipeline = Pipeline([('model', KerasRegressor(build_fn=m, nb_epoch=100, batch_size=100, verbose=1))])
    results = cross_val_score(pipeline, X, y, cv=kfold)
    print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

W1008 23:46:31.036391 140499308492608 deprecation_wrapper.py:119] From /home/pawel/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/1
200/200 [==============================] - 0s 229us/step
Epoch 1/1
200/200 [==============================] - 0s 306us/step
Epoch 1/1
200/200 [==============================] - 0s 341us/step
Epoch 1/1
200/200 [==============================] - 0s 396us/step
Epoch 1/1
200/200 [==============================] - 0s 463us/step
Results: -148539.75 (62724.39) MSE
Epoch 1/1
200/200 [==============================] - 0s 577us/step
Epoch 1/1
200/200 [==============================] - 0s 675us/step
Epoch 1/1
200/200 [==============================] - 0s 797us/step
Epoch 1/1
200/200 [==============================] - 0s 872us/step
Epoch 1/1
200/200 [==============================] - 0s 969us/step
Results: -145979.10 (61842.24) MSE
Epoch 1/1
200/200 [==============================] - 0s 1ms/step
Epoch 1/1
200/200 [==============================] - 0s 1ms/step
Epoch 1/1
200/200 [==============================] - 0s 1ms/step
Epoch 1/1
200/200 [==============================] - 0s 1ms/step


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=100)
model = gen_model_2()
model.fit(X_train, y_train, epochs=5000, callbacks=[early_stopping])

Epoch 1/5000
670/670 [==============================] - 1s 1ms/step - loss: 133603.5332
Epoch 2/5000
670/670 [==============================] - 0s 156us/step - loss: 36139.6377
Epoch 3/5000
670/670 [==============================] - 0s 187us/step - loss: 21325.7121
Epoch 4/5000
670/670 [==============================] - 0s 191us/step - loss: 20191.7130
Epoch 5/5000
670/670 [==============================] - 0s 159us/step - loss: 19476.7813
Epoch 6/5000
670/670 [==============================] - 0s 184us/step - loss: 19458.9540
Epoch 7/5000
670/670 [==============================] - 0s 183us/step - loss: 18929.1995
Epoch 8/5000
670/670 [==============================] - 0s 178us/step - loss: 18657.4419
Epoch 9/5000
670/670 [==============================] - 0s 184us/step - loss: 18412.2042
Epoch 10/5000
670/670 [==============================] - 0s 189us/step - loss: 18140.2367
Epoch 11/5000
670/670 [==============================] - 0s 181us/step - loss: 18046.9572
Epoch 12/5000
670/67

670/670 [==============================] - 0s 172us/step - loss: 15800.0150
Epoch 93/5000
670/670 [==============================] - 0s 171us/step - loss: 16005.6828
Epoch 94/5000
670/670 [==============================] - 0s 159us/step - loss: 16119.4790
Epoch 95/5000
670/670 [==============================] - 0s 175us/step - loss: 15856.6337
Epoch 96/5000
670/670 [==============================] - 0s 178us/step - loss: 15884.4182
Epoch 97/5000
670/670 [==============================] - 0s 182us/step - loss: 15898.8131
Epoch 98/5000
670/670 [==============================] - 0s 167us/step - loss: 16617.1921
Epoch 99/5000
670/670 [==============================] - 0s 159us/step - loss: 16115.6144
Epoch 100/5000
670/670 [==============================] - 0s 162us/step - loss: 15789.7470
Epoch 101/5000
670/670 [==============================] - 0s 189us/step - loss: 16107.0047
Epoch 102/5000
670/670 [==============================] - 0s 166us/step - loss: 16010.7880
Epoch 103/5000
670/67

670/670 [==============================] - 0s 161us/step - loss: 12212.7019
Epoch 183/5000
670/670 [==============================] - 0s 176us/step - loss: 12035.1815
Epoch 184/5000
670/670 [==============================] - 0s 156us/step - loss: 11993.1329
Epoch 185/5000
670/670 [==============================] - 0s 162us/step - loss: 12045.6261
Epoch 186/5000
670/670 [==============================] - 0s 147us/step - loss: 11800.5655
Epoch 187/5000
670/670 [==============================] - 0s 166us/step - loss: 11820.1002
Epoch 188/5000
670/670 [==============================] - 0s 185us/step - loss: 11708.3145
Epoch 189/5000
670/670 [==============================] - 0s 189us/step - loss: 11663.6758
Epoch 190/5000
670/670 [==============================] - 0s 161us/step - loss: 11452.4123
Epoch 191/5000
670/670 [==============================] - 0s 161us/step - loss: 11397.9326
Epoch 192/5000
670/670 [==============================] - 0s 160us/step - loss: 11748.3079
Epoch 193/5000

670/670 [==============================] - 0s 168us/step - loss: 8237.8497
Epoch 273/5000
670/670 [==============================] - 0s 172us/step - loss: 8135.6706
Epoch 274/5000
670/670 [==============================] - 0s 193us/step - loss: 8017.2402
Epoch 275/5000
670/670 [==============================] - 0s 170us/step - loss: 7675.9875
Epoch 276/5000
670/670 [==============================] - 0s 161us/step - loss: 7976.8203
Epoch 277/5000
670/670 [==============================] - 0s 187us/step - loss: 7764.7098
Epoch 278/5000
670/670 [==============================] - 0s 177us/step - loss: 7691.7196
Epoch 279/5000
670/670 [==============================] - 0s 181us/step - loss: 8007.0597
Epoch 280/5000
670/670 [==============================] - 0s 178us/step - loss: 8032.5213
Epoch 281/5000
670/670 [==============================] - 0s 168us/step - loss: 7790.3527
Epoch 282/5000
670/670 [==============================] - 0s 181us/step - loss: 7540.0109
Epoch 283/5000
670/670 [=

670/670 [==============================] - 0s 185us/step - loss: 4687.6878
Epoch 364/5000
670/670 [==============================] - 0s 170us/step - loss: 4326.9204
Epoch 365/5000
670/670 [==============================] - 0s 158us/step - loss: 4291.3350
Epoch 366/5000
670/670 [==============================] - 0s 173us/step - loss: 4443.8705
Epoch 367/5000
670/670 [==============================] - 0s 151us/step - loss: 4155.0447
Epoch 368/5000
670/670 [==============================] - 0s 169us/step - loss: 4234.9431
Epoch 369/5000
670/670 [==============================] - 0s 182us/step - loss: 4174.4571
Epoch 370/5000
670/670 [==============================] - 0s 177us/step - loss: 4356.0033
Epoch 371/5000
670/670 [==============================] - 0s 187us/step - loss: 4812.6735
Epoch 372/5000
670/670 [==============================] - 0s 166us/step - loss: 4488.1115
Epoch 373/5000
670/670 [==============================] - 0s 155us/step - loss: 4259.2103
Epoch 374/5000
670/670 [=

...

670/670 [==============================] - 0s 149us/step - loss: 248.5110
Epoch 1913/5000
670/670 [==============================] - 0s 155us/step - loss: 215.8202
Epoch 1914/5000
670/670 [==============================] - 0s 168us/step - loss: 201.9011
Epoch 1915/5000
670/670 [==============================] - 0s 170us/step - loss: 155.9204
Epoch 1916/5000
670/670 [==============================] - 0s 154us/step - loss: 125.8002
Epoch 1917/5000
670/670 [==============================] - 0s 167us/step - loss: 152.9856
Epoch 1918/5000
670/670 [==============================] - 0s 160us/step - loss: 141.8313
Epoch 1919/5000
670/670 [==============================] - 0s 161us/step - loss: 115.7125
Epoch 1920/5000
670/670 [==============================] - 0s 163us/step - loss: 89.4797
Epoch 1921/5000
670/670 [==============================] - 0s 153us/step - loss: 99.8608
Epoch 1922/5000
670/670 [==============================] - 0s 176us/step - loss: 160.8865
Epoch 1923/5000
670/670 [===

670/670 [==============================] - 0s 176us/step - loss: 260.0407
Epoch 2004/5000
670/670 [==============================] - 0s 165us/step - loss: 188.1534
Epoch 2005/5000
670/670 [==============================] - 0s 168us/step - loss: 145.7103
Epoch 2006/5000
670/670 [==============================] - 0s 163us/step - loss: 120.5485
Epoch 2007/5000
670/670 [==============================] - 0s 162us/step - loss: 97.5476
Epoch 2008/5000
670/670 [==============================] - 0s 169us/step - loss: 132.1264
Epoch 2009/5000
670/670 [==============================] - 0s 161us/step - loss: 165.6245
Epoch 2010/5000
670/670 [==============================] - 0s 165us/step - loss: 160.6141
Epoch 2011/5000
670/670 [==============================] - 0s 166us/step - loss: 238.7633
Epoch 2012/5000
670/670 [==============================] - 0s 167us/step - loss: 168.9950
Epoch 2013/5000
670/670 [==============================] - 0s 187us/step - loss: 205.7187
Epoch 2014/5000
670/670 [==

670/670 [==============================] - 0s 176us/step - loss: 87.4315
Epoch 2095/5000
670/670 [==============================] - 0s 150us/step - loss: 83.5247
Epoch 2096/5000
670/670 [==============================] - 0s 156us/step - loss: 112.4357
Epoch 2097/5000
670/670 [==============================] - 0s 178us/step - loss: 128.2490
Epoch 2098/5000
670/670 [==============================] - 0s 158us/step - loss: 131.5494
Epoch 2099/5000
670/670 [==============================] - 0s 161us/step - loss: 97.5062
Epoch 2100/5000
670/670 [==============================] - 0s 158us/step - loss: 85.0279
Epoch 2101/5000
670/670 [==============================] - 0s 161us/step - loss: 76.9071
Epoch 2102/5000
670/670 [==============================] - 0s 157us/step - loss: 82.6993
Epoch 2103/5000
670/670 [==============================] - 0s 159us/step - loss: 91.5780
Epoch 2104/5000
670/670 [==============================] - 0s 160us/step - loss: 84.8442
Epoch 2105/5000
670/670 [=========

670/670 [==============================] - 0s 173us/step - loss: 70.3133
Epoch 2186/5000
670/670 [==============================] - 0s 168us/step - loss: 61.2015
Epoch 2187/5000
670/670 [==============================] - 0s 165us/step - loss: 59.6775
Epoch 2188/5000
670/670 [==============================] - 0s 167us/step - loss: 67.7060
Epoch 2189/5000
670/670 [==============================] - 0s 177us/step - loss: 92.3347
Epoch 2190/5000
670/670 [==============================] - 0s 169us/step - loss: 107.7666
Epoch 2191/5000
670/670 [==============================] - 0s 150us/step - loss: 83.0082
Epoch 2192/5000
670/670 [==============================] - 0s 169us/step - loss: 63.9275
Epoch 2193/5000
670/670 [==============================] - 0s 176us/step - loss: 61.9594
Epoch 2194/5000
670/670 [==============================] - 0s 173us/step - loss: 63.1153
Epoch 2195/5000
670/670 [==============================] - 0s 158us/step - loss: 68.1271
Epoch 2196/5000
670/670 [===========

670/670 [==============================] - 0s 162us/step - loss: 134.1794
Epoch 2277/5000
670/670 [==============================] - 0s 153us/step - loss: 118.1318
Epoch 2278/5000
670/670 [==============================] - 0s 156us/step - loss: 80.4826
Epoch 2279/5000
670/670 [==============================] - 0s 162us/step - loss: 88.7576
Epoch 2280/5000
670/670 [==============================] - 0s 168us/step - loss: 117.5386
Epoch 2281/5000
670/670 [==============================] - 0s 169us/step - loss: 121.1017
Epoch 2282/5000
670/670 [==============================] - 0s 168us/step - loss: 165.5104
Epoch 2283/5000
670/670 [==============================] - 0s 172us/step - loss: 182.7553
Epoch 2284/5000
670/670 [==============================] - 0s 178us/step - loss: 185.0886
Epoch 2285/5000
670/670 [==============================] - 0s 156us/step - loss: 130.9590
Epoch 2286/5000
670/670 [==============================] - 0s 162us/step - loss: 103.5685
Epoch 2287/5000
670/670 [===

In [24]:
for x in range(30):
    print(model.predict(X_train[x].reshape(1, 279)), y_train[x])

[[ 32.774265 132.81142  435.45154 ]] [ 18. 108. 432.]
[[141.407   302.32706 356.33405]] [144. 288. 288.]
[[127.04638 355.95224 671.3979 ]] [126. 360. 684.]
[[ 33.012733 247.1586   404.0207  ]] [ 18. 252. 360.]
[[ 54.256844 179.17249  217.06184 ]] [ 54. 180. 216.]
[[160.74869 517.99225 765.86487]] [162. 504. 756.]
[[103.02828 358.4464  558.10254]] [108. 360. 576.]
[[192.73305 437.9208  698.05194]] [180. 396. 684.]
[[141.10146 319.02774 338.80536]] [144. 324. 324.]
[[ 91.60093 295.60992 303.10815]] [ 90. 288. 288.]
[[ 32.882915 357.73788  457.91855 ]] [ 36. 360. 468.]
[[106.58921 523.27185 784.3856 ]] [108. 504. 756.]
[[ 19.77142 269.2325  301.55124]] [ 18. 288. 288.]
[[100.802505 428.53226  702.3547  ]] [108. 432. 720.]
[[122.337616 324.46515  624.8712  ]] [126. 288. 576.]
[[124.297295 599.94214  749.8661  ]] [126. 576. 720.]
[[ 93.413765 536.7481   748.51984 ]] [ 90. 504. 720.]
[[101.621056 389.1942   450.9047  ]] [108. 396. 468.]
[[ 23.053896  63.213478 286.58    ]] [ 36.  72. 324.]
[

In [25]:
model.save('task2.model')